  # `unittest`
**Testing with the standard library**

[docs.python.org/3/library/unittest.html](http://docs.python.org/3/library/unittest.html)

# Basic features

* Test automation
* Sharing of setup and shutdown code for tests
* Aggregation of tests into collections
* Independence of the tests from the reporting framework.

* TestCase the basic unit of test organization
 * provides assertions
 * provides fixtures
 * Groups together related test functions
* Test methods
* assertions
 * equality, truth, is/not
 * contained
 * raises, logs, warns
 * comparison operators
 * list/set/etc. equality
* fixtures
 * setUp/tearDown
 * setUpClass/tearDownClass
 * setUpModule/tearDownModule
* test discovery
* unittest.main()
* test skipping
* subtests
* TestLoader
* TestRunner
* TestResult


# `unittest.TestCase`
*The basic unit of test organization*

* provides assertion methods
* test and class fixtures
* related test methods



**To create a test**, subclass from `unittest.TestCase` and implement test methods.

In [ ]:
# This sets up some utilities that we use later in the presentation
import unittest

def run(test_case):
    """Run a test case and display basic results.
    """
    runner = unittest.defaultTestLoader.loadTestsFromTestCase(Tests)
    result = unittest.TestResult()
    result = runner.run(result)
    print('FAILURES: {}'.format(len(result.failures)))
    for f in result.failures:
        print(f[1])
    print('ERRORS: {}'.format(len(result.errors)))
    for e in result.errors:
        print(e[1])
    print('SKIPPED: {}'.format(len(result.skipped)))
    for s in result.skipped:
        print(s[1])
    print('TOTAL: {}'.format(result.testsRun))

# The simplest `TestCase` example

In [ ]:
# This TestCase does nothing
class Tests(unittest.TestCase):
    pass

# Running this will report zero tests.
run(Tests)

Not terribly exciting!

# Test methods start with "test"

Actual testing occurs in `TestCase` methods that start with "test".

In [ ]:
class Tests(unittest.TestCase):
    # test_ indicates a test to be run. This one does nothing.
    def test_empty(self):
        pass
    
    # So does just "test", but it's less Pythonic
    def testNothing(self):
        pass
    
    # This test isn't found because it doesn't start with test
    def empty_test(self):
        pass
    
run(Tests)

Still not very interesting, but we see that a test has been run!

# Assertions
*Expressions of expectations*

`TestCase` provides assertion methods that determine if tests pass or fail. 

In [ ]:
class Tests(unittest.TestCase):
    # This test passes trivially
    def test_always_passes(self):
        self.assertTrue(True)
        
    # This test fails trivially
    def test_always_fails(self):
        self.assertFalse(True)
        
run(Tests)

## Common assertions

| method name                 | checks               | | method name                | checks                 |
|-----------------------------|----------------------|-|----------------------------|------------------------|
| `assertEqual(a, b)`         | `a == b`             | | `assertIsNone(x)`	        | `x is None`            |
| `assertNotEqual(a, b)`      | `a != b`             | | `assertIsNotNone(x)`		| `x is not None`        |
| `assertTrue(x)`	          | `bool(x) is True`    | | `assertIn(a, b)`		    | `a in b`               |
| `assertFalse(x)`            | `bool(x) is False`   | | `assertNotIn(a, b)`        | `a not in b`           |
| `assertIs(a, b)`		      | `a is b`             | |`assertIsInstance(a, b)`	| `isinstance(a, b)`     |
| `assertIsNot(a, b)`		  | `a is not b`         | |`assertNotIsInstance(a, b)`	| `not isinstance(a, b)` |

See full details in the [`TestCase` documentation](https://docs.python.org/3/library/unittest.html#unittest.TestCase)

## Testing for exceptions
`assertRaises` is a *context-manager* that checks for an exception.

In [ ]:
class Tests(unittest.TestCase):
    # Verify that an IndexError is thrown in the test
    def test_index_error(self):
        with self.assertRaises(IndexError):
            raise IndexError()
            
    def test_value_error(self):
        # Verify that a ValueError is thrown in the test
        with self.assertRaises(ValueError):
            pass
            
run(Tests)

# Fixtures
*Test set-up and clean-up*

Code run at various times in the test run:

| name                     | type            | when                         |
|--------------------------|-----------------|------------------------------|
| `TestCase.setUp`         | instance method | before each test method      |
| `TestCase.tearDown`      | instance method | after each test method       |
| `TestCase.setUpClass`    | class method    | before any methods run       |
| `TestCase.tearDownClass` | class method    | after all methods run        |
| `setUpModule`            | global function | before any methods in module |
| `tearDownModule`         | global function | after all methods in module  |

## What are fixtures for?
They ensure a known state for tests and clean up resources.

* Create/clear database tables
* Configure instance members
* Create/delete temporary files

In [ ]:
class Tests(unittest.TestCase):
    # This is run once before any of the tests in the TestCase
    @classmethod
    def setUpClass(cls): print('set up class')
        
    # This is run once after all of the tests in the TestCase have run
    @classmethod
    def tearDownClass(cls): print('tear down class')
        
    # This is run before *each* test in the TestCase
    def setUp(self): print('- set up')
        
    # This is run after *each* test in the TestCase
    def tearDown(self): print('- tear down')
        
    # A few demo tests
    def test_one(self): print('-- test one')
    def test_two(self): print('-- test two')
    
run(Tests)

## Configuring member attributes

`setUp()` is often used to assign to attributes used in tests.

In [ ]:
class Tests(unittest.TestCase):
    def setUp(self):
        # self.data will be set to this value before each test in the TestCase
        self.data = {'a': 2}
        
    def test_foo(self):
        # We can test the value of self.data
        self.assertEqual(self.data['a'], 2)
        
        # Modifications to self.data will be reset by setUp()
        self.data['a'] = 42
        
    def test_bar(self):
        self.assertEqual(self.data['a'], 2)
        self.data['a'] = 1337
        
run(Tests)

# `unittest.main()`
*Convenience function for running from the command line*

```python
import unittest


class Tests(unittest.TestCase):
    # This fails trivially
    def test_foo(self):
        self.assertEqual(1, 2)

# This the idiomatic test for "am I the top-level program" in Python
if __name__ == '__main__':
    # If this is the top-level program, this scans for all TestCases and runs them.
    unittest.main()
```

In [ ]:
%run examples/unittest_main.py

# Skipping tests with decorators
*Use the decorators `skip()`, `skipIf`, and `skipUnless` to skip tests*

In [ ]:
# This controls which tests run
some_condition = True

class Tests(unittest.TestCase):
    @unittest.skip('This test never runs')
    def test_no_run(self):
        print('This will never execute in a test suite.')
        
    # We only run this test if our condition is False
    @unittest.skipIf(some_condition == True, 'Condition is true')
    def test_when_condition_is_false(self):
        pass
    
    # We only run this test if our condition is True
    @unittest.skipUnless(some_condition == True, 'Condition is false')
    def test_when_condition_is_true(self):
        pass
    
run(Tests)

## Skipping tests from fixtures with `SkipTest`
* Throw the `SkipTest` exception from `setUp()` to skip tests

In [ ]:
class Tests(unittest.TestCase):
    def setUp(self):
        # We skip tests if a necessary resource is not available
        raise unittest.SkipTest(
            'Satellite uplink is not available right now.')
        
    # This test will only be run if the satellite uplink is available
    def test_contact_satellite(self):
        pass
    
run(Tests)

# Allow failures with `expectedFailure`
*`expectedFailure` allows a failing test to count as passing*

In [ ]:
class Tests(unittest.TestCase):
    # The failing assertion in this test is not counted as a failure
    @unittest.expectedFailure
    def test_this_will_fail(self):
        self.assertTrue(False)
        
run(Tests)

# Parameterize tests with `TestCase.subTest`
*Run the same test with multiple parameters, treating each as a separate test*
*New in 3.4*

This reduces boilerplate in certain situations since failure of one subtest does not prevent execution of other subtests.

In [ ]:
import string

class Tests(unittest.TestCase):
    def test_all_ascii(self):
        for word in ['thread', 'tråd', 'island', 'øy']:
            with self.subTest(word=word):
                self.assertTrue(all(c in string.ascii_lowercase for c in word))
            
run(Tests)